In [ ]:
from motor.motor_asyncio import AsyncIOMotorClient

# DB 연결 클라이언트
# 인자로는 데이터베이스의 연결 문자열
db_client = AsyncIOMotorClient("")

# myGPT 데이터 베이스를 쓰도록 설정
db = db_client['myGPT']


# 데이터 넣기
# 지정힌DB.테이블.insert_one( {JSON 데이터} )
# await db.messages.insert_one({"channel_id": "asdfasdf", "content": "Data_02" })



In [ ]:

# 데이터 조회
# 조회는 기본적으로 _id 를 기준으로 오래된 것부터 나옴
await db.messages.find({"channel_id": "asdfasdf"}).to_list()



In [ ]:
# .aio : 비동기 클라이언트
from azure.servicebus.aio import ServiceBusClient
# ServiceBusMessage : 큐에 보낼 메세지의 형식. 큐에 메세지 보낼 때 사용
from azure.servicebus import ServiceBusMessage
import json

# 연결 문자열을 받고 해야할 것
#   환경변수 세팅. local.settings.json 에서 DB_CONNECTION_URLR 값 입력
#   클라이언트를 생성, 해당 클라이언트를 통해서 큐에 대한 sender와 receiver 를 생성 가능
#       sender : 큐에 메세지를 보내는 인스턴스
#       receiver : 큐에 메세지를 받는 인스턴스

SERVICEBUS_CONNECTION_URL = ""

# 클라이언트 생성
# ServiceBusClient.from_connection_string(conn_str={서비스버스 연결 문자열}, logging_enable=True (로깅 활성화))
servicebus_client = ServiceBusClient.from_connection_string(conn_str=SERVICEBUS_CONNECTION_URL, logging_enable=True)

# 큐에 보낼 임시 메세지
question_json_data = {
    "channel_id": "asdfasdf",
    "content": "DataDATATATAT"
}


# 큐에 메세지를 보내는 sender 생성
# 비동기

# 컨텍스트 관리자 열기?
# with 문법 사용 이유 : servicebus_client 내부에 있는 __aenter__ 와 __aexit__ 메서드가 알아서 호출됨
# = 자원의 낭비 없이 코드 작성 가능
async with servicebus_client:
    # 큐에 메세지를 보내는 sender 생성
    # get_queue_sender : 직접적으로 사용할 큐 이름을 지정
    sender = servicebus_client.get_queue_sender(queue_name="process-request-queue")

    # sender 를 열기 (얘도 with 문법 사용)
    async with sender:
        # 보낼 메세지를 ServiceBusMessage 형식으로 변환
        # json 형식이 없으므로, json.dumps() 함수를 사용
        message = ServiceBusMessage(json.dumps(question_json_data))
        
        # 큐에 메세지 보내기
        await sender.send_messages(message)

    # With 문법을 사용했으므로 작업 완료 시 sender.close(), servicebus_client.close() 를 호출할 필요 없음


